In [7]:
import glob

import tqdm

import collections

#### Analysing the results from the algorithm

In [2]:
files = glob.glob("../data/set_cover_output/*.txt")

In [3]:
len(files)

68980

In [6]:
smallest_sets = []

info = {}

count = 0

for file in tqdm.tqdm_notebook(files):
    with open(file) as f:

        lines = f.readlines()

        seed = int(file.split("seed_")[-1].split(".txt")[0])

        starting_size = int(file.split("starting_size_")[-1].split("_")[0])

        starting_index = int(file.split("starting_index_")[-1].split("_")[0])


        info[count] = {"seed": seed, 
                       "starting_size": starting_size,
                       "starting_index": starting_index,
                       "indices": lines[0].split(",")[2:],
                       "len": len(lines[0].split(",")[2:])}


        count += 1

<ipython-input-6-1ff89b22bd03>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm.tqdm_notebook(files):


In [8]:
collections.Counter([k['len'] for k in info.values()])

Counter({13: 19856, 12: 28385, 14: 3899, 11: 14876, 10: 1964})

### Numerical evaluation for memory-two strategies.

In [18]:
def seperate_equilibria_and_non(df):
    per_player = df.groupby('ID')['condition A'].all()

    eq_players_ID = per_player[per_player == True].index

    noneq_players_ID = per_player[per_player == False].index
    
    return df[df['ID'].isin(eq_players_ID)], df[df['ID'].isin(noneq_players_ID)]

In [9]:
import os

from importlib.machinery import SourceFileLoader

os.chdir('../')

main = SourceFileLoader("main", "src/main.py").load_module()

from main import *

eq = SourceFileLoader("eq", "src/numerical_equilibria_n_bit_vs_n_bit.py").load_module()

from eq import *

os.chdir("nbs/")

In [24]:
import pandas

from collections import Counter

In [13]:
columns = (
    ["", "ID"]
    + [f"p{i+1}" for i in range(16)]
    + ["label", "Sp", "Sq", "condition A", "condition B", "c", "b"]
)

In [16]:
files = glob.glob("../data/memory_two_against_memory_two/*.csv")

In [17]:
dfs = [pd.read_csv(file, index_col=0, names=columns) for file in files]

df = pd.concat(dfs)

In [19]:
eqmemtwo, noneqmemtwo = seperate_equilibria_and_non(df)

In [20]:
groups = noneqmemtwo[noneqmemtwo['condition A'] == False].groupby('ID')

sets, ids = [], []

for i, group in groups:
    sets.append(group['label'].values)
    ids.append(group['ID'].unique())

In [21]:
len(sets)

535

In [22]:
flat_list = [item for sublist in sets for item in sublist]

In [25]:
counter = Counter(flat_list)

### Check the conditions 

In [40]:
def mem_two_condition_one(p, b, c):
    return (p[10] < p[6]) 

def mem_two_condition_two(p, b, c):
    return c * p[11] + b * (-1 + p[6]) < 0


def mem_two_condition_three(p, b, c):
    return c * (1 + p[12] - p[6])  + b * (-1 + p[6])


def mem_two_condition_four(p, b, c):
    return (p[13] < 1)


def mem_two_condition_five(p, b, c):
    return c * p[15] + b * (-1 + p[6]) < 0


def mem_two_condition_six(p, b, c):
    return c * p[3] + b * (-1 + p[6]) < 0


def mem_two_condition_seven(p, b, c):
    return (c * (1 + p[15] - p[6]) + b * (-1 + p[6])) < 0 


def mem_two_condition_eight(p, b, c):
    return (b * (-1 + p[6]) + c * p[7]) < 0 


def mem_two_condition_nine(p, b, c):
    return ((b - c) * (-1 + p[6]) + c * p[8]) < 0 


def mem_two_condition_ten(p, b, c):
    return (p[14] < p[6])


def mem_two_condition_eleven(p, b, c):
    return (p[2] < p[6])


def mem_two_condition_twelve(p, b, c):
    return p[5] < 1


def mem_two_condition_thirteen(p, b, c):
    return (p[9] < 1)

In [46]:
def is_Nash(p, b, c):
    return { "cond. 1": mem_two_condition_one(p, b, c),
             "cond. 2": mem_two_condition_two(p, b, c),
             "cond. 3": mem_two_condition_three(p, b, c),
             "cond. 4": mem_two_condition_four(p, b, c),
             "cond. 5": mem_two_condition_five(p, b, c),
             "cond. 6": mem_two_condition_six(p, b, c),
             "cond. 7": mem_two_condition_seven(p, b, c),
             "cond. 8": mem_two_condition_eight(p, b, c),
             "cond. 9": mem_two_condition_nine(p, b, c),
             "cond. 10": mem_two_condition_ten(p, b, c),
             "cond. 11": mem_two_condition_eleven(p, b, c),
             "cond. 12": mem_two_condition_twelve(p, b, c),
             "cond. 13": mem_two_condition_thirteen(p, b, c)}

In [47]:
b, c = 2, 1

In [48]:
eqmemtwo = eqmemtwo.drop_duplicates("ID")

In [49]:
players_cols =  [f"p{i+1}" for i in range(16)]

In [50]:
outside = 0

for i, row in eqmemtwo[players_cols].drop_duplicates().iterrows():
    p = row[players_cols]
    
    check = all(is_Nash(p, b, c).values())
    
    if check == False:
        outside += 1

In [52]:
outside

380

In [53]:
eqmemtwo['ID'].nunique()

387